In [3]:

!pip install numpy pandas librosa scipy tqdm scikit-learn imbalanced-learn torch torchaudio transformers jiwer pystoi nltk



Defaulting to user installation because normal site-packages is not writeable


In [4]:
import os
import numpy as np
import pandas as pd
import librosa
from scipy import signal
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from abc import ABC, abstractmethod
import os
import numpy as np
import pandas as pd
import librosa
from scipy import signal
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from abc import ABC, abstractmethod

import torch
import torchaudio
import os
import numpy as np
from tqdm import tqdm
import pandas as pd
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from jiwer import wer
from pystoi import stoi
import nltk
from nltk.corpus import cmudict
from scipy import signal
import librosa
from sklearn.decomposition import PCA



In [5]:
import os
import numpy as np
import pandas as pd
import librosa
from scipy import signal
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from abc import ABC, abstractmethod


class AudioFeatureExtractor(ABC):
    def __init__(self, n_mfcc=20, max_pad_length=1000):
        self.sr = None
        self.n_mfcc = n_mfcc
    
    def extract_mfcc(self, processed_audio):
        window_size = int(0.5 * self.sr)
        step_size = int(0.25 * self.sr)
        mfcc_list = []
    
        for start in range(0, len(processed_audio) - window_size + 1, step_size):
            end = start + window_size
            window = processed_audio[start:end]
            
            if len(window) != window_size:
                continue
    
            mfcc = librosa.feature.melspectrogram(y=window, sr=self.sr)
            mfcc_list.append(mfcc.mean(axis=1))
    
        return mfcc_list
        
    @abstractmethod
    def process_audio(self, audio):
        pass
    
    def extract_features(self, audio_path):
        try:
            audio, sr = librosa.load(audio_path)
            self.sr = sr
            processed_audio = self.process_audio(audio)
            return self.extract_mfcc(processed_audio)
        except Exception as e:
            return None

<h1> Binary Classification </h1>

In [13]:
import os
import numpy as np
import pandas as pd
import librosa
from scipy import signal
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from abc import ABC, abstractmethod
    

class AudioFeatureExtractor(ABC):
    def __init__(self, n_mfcc=20, max_pad_length=1000):
        self.sr = None
        self.n_mfcc = n_mfcc
    
    def extract_mfcc(self, processed_audio):
        window_size = int(0.5 * self.sr)
        step_size = int(0.25 * self.sr)
        mfcc_list = []
    
        for start in range(0, len(processed_audio) - window_size + 1, step_size):
            end = start + window_size
            window = processed_audio[start:end]
            
            if len(window) != window_size:
                continue
    
            mfcc = librosa.feature.melspectrogram(y=window, sr=self.sr)
            mfcc_list.append(mfcc.mean(axis=1))
    
        return mfcc_list
        
    @abstractmethod
    def process_audio(self, audio):
        pass
    
    def extract_features(self, audio_path):
        try:
            audio, sr = librosa.load(audio_path)
            self.sr = sr
            processed_audio = self.process_audio(audio)
            return self.extract_mfcc(processed_audio)
        except Exception as e:
            return None
        
import os
import json
import torch
import numpy as np

from init_config import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class LogisticRegressionClassifier(torch.nn.Module):
    def __init__(self, feature_dim=129):
        super(LogisticRegressionClassifier, self).__init__()
        self.linear1 = torch.nn.Linear(feature_dim, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, xx):
        return self.sigmoid(self.linear1(torch.nn.functional.normalize(xx, p=1.0, dim=1)))


def load_kirigami_model():
    # Phoneme Model!
    my_phoneme_filter_model = LogisticRegressionClassifier(feature_dim=129)
    my_phoneme_filter_model.load_state_dict(
    torch.load(lr_phoneme_checkpoint_path, map_location=torch.device('cpu')))
    my_phoneme_filter_model.eval()

    return my_phoneme_filter_model


def load_background_filter_model():
    # Background Model!
    my_background_filter_model = LogisticRegressionClassifier(feature_dim=129)
    my_background_filter_model.load_state_dict(torch.load(bg_lr_checkpoint_path, map_location=torch.device('cpu')))
    my_background_filter_model.eval()

    return my_background_filter_model


def kirigami_filter_torch(s_full, threshold=0.5):
    lr_phoneme_filter_model = LogisticRegressionClassifier(feature_dim=129)
    # load the model if in kirigami_filters directory
    if os.path.exists("./kirigami_filters/scipy_phoneme_filter.ckpt"):
        lr_phoneme_filter_model.load_state_dict(
            torch.load("./kirigami_filters/scipy_phoneme_filter.ckpt", map_location=device))
    else:
        raise FileNotFoundError("Phoneme filter model not found")

    lr_phoneme_filter_model.eval()
    pred = (lr_phoneme_filter_model.forward(torch.Tensor(s_full)) >= threshold).long().numpy()
    masked = (1 - pred) * s_full
    return masked


def kirigami_filter(stft):
    output_sp = np.zeros_like(stft)
    for i, fft in enumerate(stft):

        sum = np.sum(fft)

        product = 0
        for iw, (vv, ww) in enumerate(zip(fft, weight)):
            product = product + vv * weight[iw]
        product = product / sum
        product = product + bias

        z = 1 / (1 + np.exp(-product))
        # print("LR filter probability", i, z)
        if z < LR_THRESHOLD:
            # add the value
            output_sp[i] = stft[i]
    return output_sp


def kirigami_filter_reverse_fft(stft, stft_original):
    output_sp = np.zeros_like(stft)
    for i, fft in enumerate(stft):
        sum = np.sum(fft)
        product = 0
        for iw, (vv, ww) in enumerate(zip(fft, weight)):
            product = product + vv * weight[iw]
        product = product / sum
        product = product + bias
        z = 1 / (1 + np.exp(-product))
        # print("LR filter probability", i, z)
        if z < LR_THRESHOLD:
            # add the value
            # output_sp[i] = stft[i]
            output_sp[i] = stft_original[i]
    return output_sp


def background_detection_filter(stft):
    output_sp = np.zeros_like(stft)
    for i, fft in enumerate(stft):
        sum = np.sum(fft)
        product = 0
        for iw, (vv, ww) in enumerate(zip(fft, weight_background)):
            product = product + vv * weight_background[iw]
        product = product // sum
        product = product + bias_background
        z = 1 / (1 + np.exp(-product))
        # print("Background probability: ", i, z)
        if z < BACKGROUND_LR_THRESHOLD:  # lower than threshold not background.
            # add the value
            output_sp[i] = stft[i]
    return output_sp


import os
import librosa
import numpy as np
import torch
import soundfile as sf
import matplotlib.pyplot as plt

# Load Kirigami models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_kirigami_model():
    """Load the phoneme filter model."""
    model = LogisticRegressionClassifier(feature_dim=129)
    model.load_state_dict(torch.load("kirigami_filters/model_checkpoints/scipy_phoneme_filter.ckpt", map_location=device))
    model.eval()
    return model

def load_background_filter_model():
    """Load the background noise filter model."""
    model = LogisticRegressionClassifier(feature_dim=129)
    model.load_state_dict(torch.load("kirigami_filters/model_checkpoints/noisy_background_scipy_detector.ckpt", map_location=device))
    model.eval()
    return model

# Load models
phoneme_filter_model = load_kirigami_model()
background_filter_model = load_background_filter_model()

# Extract model weights
weight_phoneme = phoneme_filter_model.linear1.weight.data[0].numpy()
bias_phoneme = phoneme_filter_model.linear1.bias.data[0].numpy()
weight_background = background_filter_model.linear1.weight.data[0].numpy()
bias_background = background_filter_model.linear1.bias.data[0].numpy()


class KirigamiExtractor(AudioFeatureExtractor):
    def apply_kirigami_filter(self, stft, weight, bias, threshold=0.3):
        """Apply Kirigami logistic regression filter on STFT features (ensuring 129 features per frame)."""
        output_stft = np.zeros_like(stft)

        for i, frame in enumerate(stft):
            frame = frame[:129]  # Ensure exactly 129 dimensions

            sum_val = np.sum(frame) + 1e-6  # Avoid division by zero
            product = np.dot(frame, weight) / sum_val + bias
            prob = 1 / (1 + np.exp(-product))  # Sigmoid activation

            if prob < threshold:  # If probability is low, keep frame
                output_stft[i, :129] = frame  # Apply only to the valid region

        return output_stft

    # Process Audio File
    def process_audio(self, audio, sr=16000, threshold=0.5):
        """Process an audio file through Kirigami models and save the filtered output."""
        # Load audio
        # audio, sr = librosa.load(input_audio_path, sr=16000)

        # Compute STFT (Ensure output has 129 feature bins)
        stft = np.abs(librosa.stft(audio, n_fft=256, hop_length=128))[:129, :].T  # Transpose for correct shape

        # Apply Kirigami phoneme & background filters
        filtered_stft_phoneme = self.apply_kirigami_filter(stft, weight_phoneme, bias_phoneme, threshold)
        # filtered_stft_background = self.apply_kirigami_filter(filtered_stft_phoneme, weight_background, bias_background, threshold)

        # Convert back to audio using inverse STFT
        filtered_audio = librosa.istft(filtered_stft_phoneme.T, hop_length=128)

        return filtered_audio


import os
import numpy as np
import pandas as pd
import librosa
from scipy import signal
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from abc import ABC, abstractmethod


class RawAudioExtractor(AudioFeatureExtractor):
    def process_audio(self, audio):
        return audio

class SyntheticSensorExtractor(AudioFeatureExtractor):
    def __init__(self, window_length=256, hop_length=128, samples_per_window=10):
        super().__init__()
        self.window_length = window_length
        self.hop_length = hop_length
        self.samples_per_window = samples_per_window

    def process_audio(self, audio):
        frames = [
            audio[i:i + self.window_length]
            for i in range(0, len(audio) - self.window_length + 1, self.hop_length)
        ]
        
        processed_audio = np.zeros(len(audio))
        
        for i, frame in enumerate(frames):
            fft = np.fft.fft(frame, n=self.window_length)
            bins = np.array_split(fft, self.samples_per_window)
            reduced_fft = np.array([np.mean(bin) for bin in bins])
            
            reconstructed_fft = np.zeros_like(fft, dtype=np.complex_)
            step = len(fft) // len(reduced_fft)
            
            for j, val in enumerate(reduced_fft):
                reconstructed_fft[j * step:(j + 1) * step] = val
            
            processed_frame = np.fft.ifft(reconstructed_fft).real
            start = i * self.hop_length
            end = min(start + self.window_length, len(processed_audio))
            processed_audio[start:end] += processed_frame[:end - start]
        
        if np.max(np.abs(processed_audio)) > 0:
            processed_audio = processed_audio / np.max(np.abs(processed_audio))
            
        return processed_audio

class PrivacyMicExtractor(AudioFeatureExtractor):
    def process_audio(self, audio):
        S = librosa.stft(audio, n_fft=256, hop_length=128)
        frequencies = librosa.fft_frequencies(sr=self.sr, n_fft=256)
        S_filtered = np.where(frequencies[:, None] <= 300, S, 0)
        return librosa.istft(S_filtered, hop_length=128)

class CoughSenseExtractor(AudioFeatureExtractor):
    def process_audio(self, audio):
        n_fft = int(0.15 * self.sr)
        if n_fft % 2 != 0:
            n_fft += 1
        hop_length = n_fft // 2

        S = np.abs(librosa.stft(audio.astype(np.float64), n_fft=n_fft, hop_length=hop_length, window='hamming'))
        S_flattened = S.T
        
        pca = PCA(n_components=10)
        S_reduced = pca.fit_transform(S_flattened)
        S_reconstructed = pca.inverse_transform(S_reduced).T

        filtered_audio = librosa.istft(S_reconstructed, 
                                     hop_length=hop_length,
                                     win_length=n_fft,
                                     window='hamming')
        return filtered_audio

class SamosaExtractor(AudioFeatureExtractor):
    def __init__(self, window_length=0.6, hop_length=0.03):
        super().__init__()
        self.window_length = window_length
        self.hop_length = hop_length

    def process_audio(self, audio):
        subsampled_audio = signal.resample(audio, int(len(audio) * 1000 / self.sr))
        upsampled_audio = signal.resample(subsampled_audio, len(audio))
        return upsampled_audio
    



class AudioClassifier:
    def __init__(self, feature_extractor: AudioFeatureExtractor, ds_path: str, extractor_name: str):
        self.feature_extractor = feature_extractor
        self.ds_path = ds_path
        self.extractor_name = extractor_name
        self.features_path = "extracted_features"
        os.makedirs(self.features_path, exist_ok=True)

    def create_features_df(self, data):
        features_filepath = os.path.join(self.features_path, f"{self.extractor_name}_features2000.csv")
        features_list = []
        
        for _, row in tqdm(data.iterrows(), total=len(data)):
            features = self.feature_extractor.extract_features(
                os.path.join(self.ds_path, row['filename'])
            )
           
            if features is not None:
                for i in features:
                    features_list.append([row['emotion']] + i.tolist())

        feature_columns = [f"feature_{i}" for i in range(len(features_list[0])-1)]
        features_df = pd.DataFrame(features_list, columns=['emotion'] + feature_columns)
        features_df.to_csv(features_filepath, index=False)
        return features_df

    def run_classification(self, features_df, target_col, classes, title):
        
        df_filtered = features_df[features_df[target_col].isin(classes)]
        
        # Drop rows where all feature values are zero
        df_filtered = df_filtered.loc[~(df_filtered.drop([target_col], axis=1) == 0).all(axis=1)]
        
        X = df_filtered.drop([target_col], axis=1)
        y = df_filtered[target_col]
        
        smote = SMOTE(random_state=42)
        X_resampled, y_resampled = smote.fit_resample(X, y)
        
        X_train, X_test, y_train, y_test = train_test_split(
            X_resampled, y_resampled, test_size=0.2, random_state=42
        )
        
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        accuracy = model.score(X_test, y_test)
        print(f"\nAccuracy for {title}: {accuracy:.4f}")
        print(f"\nClassification Report for {title}:")
        print(classification_report(y_test, y_pred, target_names=classes))
        
        return accuracy
    

def print_accuracy_summary(accuracies):
    print("\n=== ACCURACY SUMMARY ===")
    
    for extractor, score in accuracies.items():
        print(f"\n{extractor.capitalize()}:")
        print(f"  Emotion: {score:.2%}")


def main():
    ds_path = "/work/pi_shenoy_umass_edu/sgomasta_umass_edu/Batch Files/Privacy Peserving Audio/Common Voice/bhawana/kirigami/kirigami_filters/CREMA_DATASET"  # Update with your CREMA-D audio directory

    # Create metadata for CREMA-D using the provided snippet
    crema_directory_list = os.listdir(ds_path)

    file_emotion = []
    file_path = []

    for file in crema_directory_list:
        if file.endswith(".wav"):  # Ensure only WAV files are processed
            # storing file paths
            file_path.append(os.path.join(ds_path, file))
            # storing file emotions
            part = file.split('_')
            if part[2] == 'SAD':
                file_emotion.append('sad')
            elif part[2] == 'ANG':
                file_emotion.append('angry')
            elif part[2] == 'DIS':
                file_emotion.append('disgust')
            elif part[2] == 'FEA':
                file_emotion.append('fear')
            elif part[2] == 'HAP':
                file_emotion.append('happy')
            elif part[2] == 'NEU':
                file_emotion.append('neutral')
            else:
                file_emotion.append('Unknown')

    # dataframe for emotion of files
    emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

    # dataframe for path of files
    path_df = pd.DataFrame(file_path, columns=['Path'])
    data = pd.concat([emotion_df, path_df], axis=1)

    # Filter for happy and sad emotions
    data = data[
        (data['Emotions'].isin(['happy', 'sad']))
    ]

    # Rename columns to match expected 'filename' and 'emotion'
    data = data.rename(columns={'Path': 'filename', 'Emotions': 'emotion'})

    # Select up to 2,000 files, as in the original code
    data = data[['filename', 'emotion']].dropna().head(2000)
    data['filename'] = data['filename'].apply(os.path.basename)

    extractors = {
        'kirigami': KirigamiExtractor(),
        'coughsense': CoughSenseExtractor(),
        'raw': RawAudioExtractor(),
        'synthetic': SyntheticSensorExtractor(),
        'privacy': PrivacyMicExtractor(),
        'samosa': SamosaExtractor()
    }

    accuracies = {}
    for name, extractor in extractors.items():
        print(f"\nProcessing with {name} extractor...")
        classifier = AudioClassifier(extractor, ds_path, name)
        features_df = classifier.create_features_df(data)
        
        accuracies[name] = classifier.run_classification(
            features_df, 'emotion', ['happy', 'sad'], 
            f"{name} - Emotion"
        )
    
    print_accuracy_summary(accuracies)

if __name__ == "__main__":
    main()


Processing with kirigami extractor...


100%|██████████| 2000/2000 [01:38<00:00, 20.21it/s]



Accuracy for kirigami - Emotion: 0.7499

Classification Report for kirigami - Emotion:
              precision    recall  f1-score   support

       happy       0.78      0.68      0.73      1753
         sad       0.72      0.82      0.77      1809

    accuracy                           0.75      3562
   macro avg       0.75      0.75      0.75      3562
weighted avg       0.75      0.75      0.75      3562


Processing with coughsense extractor...


100%|██████████| 2000/2000 [06:15<00:00,  5.33it/s]



Accuracy for coughsense - Emotion: 0.8605

Classification Report for coughsense - Emotion:
              precision    recall  f1-score   support

       happy       0.87      0.85      0.86      1759
         sad       0.85      0.87      0.86      1775

    accuracy                           0.86      3534
   macro avg       0.86      0.86      0.86      3534
weighted avg       0.86      0.86      0.86      3534


Processing with raw extractor...


...


Accuracy for synthetic - Emotion: 0.7887

Classification Report for synthetic - Emotion:
              precision    recall  f1-score   support

       happy       0.79      0.80      0.79      1836
         sad       0.78      0.78      0.78      1742

    accuracy                           0.79      3578
   macro avg       0.79      0.79      0.79      3578
weighted avg       0.79      0.79      0.79      3578


Processing with privacy extractor...


100%|██████████| 2000/2000 [01:24<00:00, 23.56it/s]



Accuracy for privacy - Emotion: 0.7405

Classification Report for privacy - Emotion:
              precision    recall  f1-score   support

       happy       0.79      0.64      0.71      1746
         sad       0.71      0.84      0.77      1823

    accuracy                           0.74      3569
   macro avg       0.75      0.74      0.74      3569
weighted avg       0.75      0.74      0.74      3569


Processing with samosa extractor...


100%|██████████| 2000/2000 [01:33<00:00, 21.48it/s]



Accuracy for samosa - Emotion: 0.7700

Classification Report for samosa - Emotion:
              precision    recall  f1-score   support

       happy       0.80      0.73      0.76      1836
         sad       0.74      0.81      0.77      1742

    accuracy                           0.77      3578
   macro avg       0.77      0.77      0.77      3578
weighted avg       0.77      0.77      0.77      3578


=== ACCURACY SUMMARY ===

Kirigami:
  Emotion: 74.99%

Coughsense:
  Emotion: 86.05%

Raw:
  Emotion: 80.72%

Synthetic:
  Emotion: 78.87%

Privacy:
  Emotion: 74.05%

Samosa:
  Emotion: 77.00%


<h1> Multi -Class Classification-Angry, sad,happy </h1>

In [17]:
import os
import numpy as np
import pandas as pd
import librosa
from scipy import signal
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from abc import ABC, abstractmethod
    

class AudioFeatureExtractor(ABC):
    def __init__(self, n_mfcc=20, max_pad_length=1000):
        self.sr = None
        self.n_mfcc = n_mfcc
    
    def extract_mfcc(self, processed_audio):
        window_size = int(0.5 * self.sr)
        step_size = int(0.25 * self.sr)
        mfcc_list = []
    
        for start in range(0, len(processed_audio) - window_size + 1, step_size):
            end = start + window_size
            window = processed_audio[start:end]
            
            if len(window) != window_size:
                continue
    
            mfcc = librosa.feature.melspectrogram(y=window, sr=self.sr)
            mfcc_list.append(mfcc.mean(axis=1))
    
        return mfcc_list
        
    @abstractmethod
    def process_audio(self, audio):
        pass
    
    def extract_features(self, audio_path):
        try:
            audio, sr = librosa.load(audio_path)
            self.sr = sr
            processed_audio = self.process_audio(audio)
            return self.extract_mfcc(processed_audio)
        except Exception as e:
            return None
        
import os
import json
import torch
import numpy as np

from init_config import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class LogisticRegressionClassifier(torch.nn.Module):
    def __init__(self, feature_dim=129):
        super(LogisticRegressionClassifier, self).__init__()
        self.linear1 = torch.nn.Linear(feature_dim, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, xx):
        return self.sigmoid(self.linear1(torch.nn.functional.normalize(xx, p=1.0, dim=1)))


def load_kirigami_model():
    # Phoneme Model!
    my_phoneme_filter_model = LogisticRegressionClassifier(feature_dim=129)
    my_phoneme_filter_model.load_state_dict(
    torch.load(lr_phoneme_checkpoint_path, map_location=torch.device('cpu')))
    my_phoneme_filter_model.eval()

    return my_phoneme_filter_model


def load_background_filter_model():
    # Background Model!
    my_background_filter_model = LogisticRegressionClassifier(feature_dim=129)
    my_background_filter_model.load_state_dict(torch.load(bg_lr_checkpoint_path, map_location=torch.device('cpu')))
    my_background_filter_model.eval()

    return my_background_filter_model


def kirigami_filter_torch(s_full, threshold=0.5):
    lr_phoneme_filter_model = LogisticRegressionClassifier(feature_dim=129)
    # load the model if in kirigami_filters directory
    if os.path.exists("./kirigami_filters/scipy_phoneme_filter.ckpt"):
        lr_phoneme_filter_model.load_state_dict(
            torch.load("./kirigami_filters/scipy_phoneme_filter.ckpt", map_location=device))
    else:
        raise FileNotFoundError("Phoneme filter model not found")

    lr_phoneme_filter_model.eval()
    pred = (lr_phoneme_filter_model.forward(torch.Tensor(s_full)) >= threshold).long().numpy()
    masked = (1 - pred) * s_full
    return masked


def kirigami_filter(stft):
    output_sp = np.zeros_like(stft)
    for i, fft in enumerate(stft):

        sum = np.sum(fft)

        product = 0
        for iw, (vv, ww) in enumerate(zip(fft, weight)):
            product = product + vv * weight[iw]
        product = product / sum
        product = product + bias

        z = 1 / (1 + np.exp(-product))
        # print("LR filter probability", i, z)
        if z < LR_THRESHOLD:
            # add the value
            output_sp[i] = stft[i]
    return output_sp


def kirigami_filter_reverse_fft(stft, stft_original):
    output_sp = np.zeros_like(stft)
    for i, fft in enumerate(stft):
        sum = np.sum(fft)
        product = 0
        for iw, (vv, ww) in enumerate(zip(fft, weight)):
            product = product + vv * weight[iw]
        product = product / sum
        product = product + bias
        z = 1 / (1 + np.exp(-product))
        # print("LR filter probability", i, z)
        if z < LR_THRESHOLD:
            # add the value
            # output_sp[i] = stft[i]
            output_sp[i] = stft_original[i]
    return output_sp


def background_detection_filter(stft):
    output_sp = np.zeros_like(stft)
    for i, fft in enumerate(stft):
        sum = np.sum(fft)
        product = 0
        for iw, (vv, ww) in enumerate(zip(fft, weight_background)):
            product = product + vv * weight_background[iw]
        product = product // sum
        product = product + bias_background
        z = 1 / (1 + np.exp(-product))
        # print("Background probability: ", i, z)
        if z < BACKGROUND_LR_THRESHOLD:  # lower than threshold not background.
            # add the value
            output_sp[i] = stft[i]
    return output_sp


import os
import librosa
import numpy as np
import torch
import soundfile as sf
import matplotlib.pyplot as plt

# Load Kirigami models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_kirigami_model():
    """Load the phoneme filter model."""
    model = LogisticRegressionClassifier(feature_dim=129)
    model.load_state_dict(torch.load("kirigami_filters/model_checkpoints/scipy_phoneme_filter.ckpt", map_location=device))
    model.eval()
    return model

def load_background_filter_model():
    """Load the background noise filter model."""
    model = LogisticRegressionClassifier(feature_dim=129)
    model.load_state_dict(torch.load("kirigami_filters/model_checkpoints/noisy_background_scipy_detector.ckpt", map_location=device))
    model.eval()
    return model

# Load models
phoneme_filter_model = load_kirigami_model()
background_filter_model = load_background_filter_model()

# Extract model weights
weight_phoneme = phoneme_filter_model.linear1.weight.data[0].numpy()
bias_phoneme = phoneme_filter_model.linear1.bias.data[0].numpy()
weight_background = background_filter_model.linear1.weight.data[0].numpy()
bias_background = background_filter_model.linear1.bias.data[0].numpy()


class KirigamiExtractor(AudioFeatureExtractor):
    def apply_kirigami_filter(self, stft, weight, bias, threshold=0.3):
        """Apply Kirigami logistic regression filter on STFT features (ensuring 129 features per frame)."""
        output_stft = np.zeros_like(stft)

        for i, frame in enumerate(stft):
            frame = frame[:129]  # Ensure exactly 129 dimensions

            sum_val = np.sum(frame) + 1e-6  # Avoid division by zero
            product = np.dot(frame, weight) / sum_val + bias
            prob = 1 / (1 + np.exp(-product))  # Sigmoid activation

            if prob < threshold:  # If probability is low, keep frame
                output_stft[i, :129] = frame  # Apply only to the valid region

        return output_stft

    # Process Audio File
    def process_audio(self, audio, sr=16000, threshold=0.5):
        """Process an audio file through Kirigami models and save the filtered output."""
        # Load audio
        # audio, sr = librosa.load(input_audio_path, sr=16000)

        # Compute STFT (Ensure output has 129 feature bins)
        stft = np.abs(librosa.stft(audio, n_fft=256, hop_length=128))[:129, :].T  # Transpose for correct shape

        # Apply Kirigami phoneme & background filters
        filtered_stft_phoneme = self.apply_kirigami_filter(stft, weight_phoneme, bias_phoneme, threshold)
        # filtered_stft_background = self.apply_kirigami_filter(filtered_stft_phoneme, weight_background, bias_background, threshold)

        # Convert back to audio using inverse STFT
        filtered_audio = librosa.istft(filtered_stft_phoneme.T, hop_length=128)

        return filtered_audio


import os
import numpy as np
import pandas as pd
import librosa
from scipy import signal
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from abc import ABC, abstractmethod


class RawAudioExtractor(AudioFeatureExtractor):
    def process_audio(self, audio):
        return audio

class SyntheticSensorExtractor(AudioFeatureExtractor):
    def __init__(self, window_length=256, hop_length=128, samples_per_window=10):
        super().__init__()
        self.window_length = window_length
        self.hop_length = hop_length
        self.samples_per_window = samples_per_window

    def process_audio(self, audio):
        frames = [
            audio[i:i + self.window_length]
            for i in range(0, len(audio) - self.window_length + 1, self.hop_length)
        ]
        
        processed_audio = np.zeros(len(audio))
        
        for i, frame in enumerate(frames):
            fft = np.fft.fft(frame, n=self.window_length)
            bins = np.array_split(fft, self.samples_per_window)
            reduced_fft = np.array([np.mean(bin) for bin in bins])
            
            reconstructed_fft = np.zeros_like(fft, dtype=np.complex_)
            step = len(fft) // len(reduced_fft)
            
            for j, val in enumerate(reduced_fft):
                reconstructed_fft[j * step:(j + 1) * step] = val
            
            processed_frame = np.fft.ifft(reconstructed_fft).real
            start = i * self.hop_length
            end = min(start + self.window_length, len(processed_audio))
            processed_audio[start:end] += processed_frame[:end - start]
        
        if np.max(np.abs(processed_audio)) > 0:
            processed_audio = processed_audio / np.max(np.abs(processed_audio))
            
        return processed_audio

class PrivacyMicExtractor(AudioFeatureExtractor):
    def process_audio(self, audio):
        S = librosa.stft(audio, n_fft=256, hop_length=128)
        frequencies = librosa.fft_frequencies(sr=self.sr, n_fft=256)
        S_filtered = np.where(frequencies[:, None] <= 300, S, 0)
        return librosa.istft(S_filtered, hop_length=128)

class CoughSenseExtractor(AudioFeatureExtractor):
    def process_audio(self, audio):
        n_fft = int(0.15 * self.sr)
        if n_fft % 2 != 0:
            n_fft += 1
        hop_length = n_fft // 2

        S = np.abs(librosa.stft(audio.astype(np.float64), n_fft=n_fft, hop_length=hop_length, window='hamming'))
        S_flattened = S.T
        
        pca = PCA(n_components=10)
        S_reduced = pca.fit_transform(S_flattened)
        S_reconstructed = pca.inverse_transform(S_reduced).T

        filtered_audio = librosa.istft(S_reconstructed, 
                                     hop_length=hop_length,
                                     win_length=n_fft,
                                     window='hamming')
        return filtered_audio

class SamosaExtractor(AudioFeatureExtractor):
    def __init__(self, window_length=0.6, hop_length=0.03):
        super().__init__()
        self.window_length = window_length
        self.hop_length = hop_length

    def process_audio(self, audio):
        subsampled_audio = signal.resample(audio, int(len(audio) * 1000 / self.sr))
        upsampled_audio = signal.resample(subsampled_audio, len(audio))
        return upsampled_audio
    

    


class AudioClassifier:
    def __init__(self, feature_extractor: AudioFeatureExtractor, ds_path: str, extractor_name: str):
        self.feature_extractor = feature_extractor
        self.ds_path = ds_path
        self.extractor_name = extractor_name
        self.features_path = "extracted_features"
        os.makedirs(self.features_path, exist_ok=True)
    

    def create_features_df(self, data):
        features_filepath = os.path.join(self.features_path, f"{self.extractor_name}_features.csv")
        features_list = []
                                         
        # Filter for happy, angry, and sad emotions during feature extraction
        filtered_data = data[data['emotion'].isin(['happy', 'angry', 'sad'])]
        print(f"Processing {len(filtered_data)} files for emotions: {filtered_data['emotion'].unique()}")
        
        for _, row in tqdm(filtered_data.iterrows(), total=len(filtered_data)):
            features = self.feature_extractor.extract_features(
                os.path.join(self.ds_path, row['filename'])
            )
           
            if features is not None:
                for i in features:
                    features_list.append([row['emotion']] + i.tolist())
                                
        if not features_list:
            raise ValueError("No features were extracted. Check if audio files are accessible or if extract_features is working correctly.")

        feature_columns = [f"feature_{i}" for i in range(len(features_list[0])-1)]
        features_df = pd.DataFrame(features_list, columns=['emotion'] + feature_columns)
        features_df.to_csv(features_filepath, index=False)
        return features_df

    def run_classification(self, features_df, target_col, classes, title):
        
        df_filtered = features_df[features_df[target_col].isin(classes)]
        
        # Drop rows where all feature values are zero
        df_filtered = df_filtered.loc[~(df_filtered.drop([target_col], axis=1) == 0).all(axis=1)]
        
        X = df_filtered.drop([target_col], axis=1)
        y = df_filtered[target_col]
        
        smote = SMOTE(random_state=42)
        X_resampled, y_resampled = smote.fit_resample(X, y)
        
        X_train, X_test, y_train, y_test = train_test_split(
            X_resampled, y_resampled, test_size=0.2, random_state=42
        )
        
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        accuracy = model.score(X_test, y_test)
        print(f"\nAccuracy for {title}: {accuracy:.4f}")
        print(f"\nClassification Report for {title}:")
        print(classification_report(y_test, y_pred, target_names=classes))
        
        return accuracy
    

def print_accuracy_summary(accuracies):
    print("\n=== ACCURACY SUMMARY ===")
    
    for extractor, score in accuracies.items():
        print(f"\n{extractor.capitalize()}:")
        print(f"  Emotion: {score:.2%}")


def main():
    ds_path = "/work/pi_shenoy_umass_edu/sgomasta_umass_edu/Batch Files/Privacy Peserving Audio/Common Voice/bhawana/CREMAD/AudioWAV"  # Updated path

    # Create metadata for CREMA-D using the provided snippet
    crema_directory_list = os.listdir(ds_path)

    print(f"\nTotal items in directory: {len(crema_directory_list)}")
    print("Sample items (first 5):", crema_directory_list[:5])

    file_emotion = []
    file_path = []

    for file in crema_directory_list:
        if file.lower().endswith(".wav"):  # Case-insensitive check
            # storing file paths
            file_path.append(os.path.join(ds_path, file))
            # storing file emotions
            part = file.split('_')
            if len(part) < 3:
                print(f"Skipping file {file}: Invalid format (less than 3 parts after splitting).")
                file_emotion.append('Unknown')
                continue
            if part[2] == 'SAD':
                file_emotion.append('sad')
            elif part[2] == 'ANG':
                file_emotion.append('angry')
            elif part[2] == 'DIS':
                file_emotion.append('disgust')
            elif part[2] == 'FEA':
                file_emotion.append('fear')
            elif part[2] == 'HAP':
                file_emotion.append('happy')
            elif part[2] == 'NEU':
                file_emotion.append('neutral')
            else:
                file_emotion.append('Unknown')

    # Debug: Check the number of WAV files processed
    print(f"\nNumber of WAV files processed: {len(file_path)}")
    print("Sample WAV files (first 5):", file_path[:5] if file_path else [])

    # dataframe for emotion of files
    emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

    # dataframe for path of files
    path_df = pd.DataFrame(file_path, columns=['Path'])
    data = pd.concat([emotion_df, path_df], axis=1)

    # Debug: Print DataFrame info before filtering
    print("\nBefore any filtering:")
    print("DataFrame shape:", data.shape)
    print("Unique emotions:", data['Emotions'].unique())
    print("Emotion counts:\n", data['Emotions'].value_counts())

    # Rename columns to match expected 'filename' and 'emotion'
    data = data.rename(columns={'Path': 'filename', 'Emotions': 'emotion'})

    # Process the entire dataset (no filtering for emotions)
    data = data[['filename', 'emotion']].dropna()
    data['filename'] = data['filename'].apply(os.path.basename)

    # Debug: Final DataFrame info
    print("\nFinal DataFrame before feature extraction:")
    print("DataFrame shape:", data.shape)
    print("Unique emotions:", data['emotion'].unique())
    print("Emotion counts:\n", data['emotion'].value_counts())

    extractors = {
        'kirigami': KirigamiExtractor(),
        'coughsense': CoughSenseExtractor(),
        'raw': RawAudioExtractor(),
        'synthetic': SyntheticSensorExtractor(),
        'privacy': PrivacyMicExtractor(),
        'samosa': SamosaExtractor()
    }

    accuracies = {}
    for name, extractor in extractors.items():
        print(f"\nProcessing with {name} extractor...")
        classifier = AudioClassifier(extractor, ds_path, name)
        features_df = classifier.create_features_df(data)
        
        accuracies[name] = classifier.run_classification(
            features_df, 'emotion', ['happy', 'angry', 'sad'], 
            f"{name} - Emotion"
        )
    
    print_accuracy_summary(accuracies)
                                         
if __name__ == "__main__":
    main()


Total items in directory: 7442
Sample items (first 5): ['1001_DFA_ANG_XX.wav', '1001_DFA_DIS_XX.wav', '1001_DFA_FEA_XX.wav', '1001_DFA_HAP_XX.wav', '1001_DFA_NEU_XX.wav']

Number of WAV files processed: 7442
Sample WAV files (first 5): ['/work/pi_shenoy_umass_edu/sgomasta_umass_edu/Batch Files/Privacy Peserving Audio/Common Voice/bhawana/CREMAD/AudioWAV/1001_DFA_ANG_XX.wav', '/work/pi_shenoy_umass_edu/sgomasta_umass_edu/Batch Files/Privacy Peserving Audio/Common Voice/bhawana/CREMAD/AudioWAV/1001_DFA_DIS_XX.wav', '/work/pi_shenoy_umass_edu/sgomasta_umass_edu/Batch Files/Privacy Peserving Audio/Common Voice/bhawana/CREMAD/AudioWAV/1001_DFA_FEA_XX.wav', '/work/pi_shenoy_umass_edu/sgomasta_umass_edu/Batch Files/Privacy Peserving Audio/Common Voice/bhawana/CREMAD/AudioWAV/1001_DFA_HAP_XX.wav', '/work/pi_shenoy_umass_edu/sgomasta_umass_edu/Batch Files/Privacy Peserving Audio/Common Voice/bhawana/CREMAD/AudioWAV/1001_DFA_NEU_XX.wav']

Before any filtering:
DataFrame shape: (7442, 2)
Unique 

100%|██████████| 3813/3813 [02:59<00:00, 21.30it/s]



Accuracy for kirigami - Emotion: 0.6463

Classification Report for kirigami - Emotion:
              precision    recall  f1-score   support

       happy       0.72      0.60      0.65      2258
       angry       0.61      0.54      0.58      2297
         sad       0.62      0.80      0.70      2202

    accuracy                           0.65      6757
   macro avg       0.65      0.65      0.64      6757
weighted avg       0.65      0.65      0.64      6757


Processing with coughsense extractor...
Processing 3813 files for emotions: ['angry' 'happy' 'sad']


 83%|████████▎ | 3170/3813 [10:32<01:51,  5.77it/s]/home/sgomasta_umass_edu/.local/lib/python3.11/site-packages/sklearn/decomposition/_pca.py:789: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var
100%|██████████| 3813/3813 [12:49<00:00,  4.96it/s]



Accuracy for coughsense - Emotion: 0.7485

Classification Report for coughsense - Emotion:
              precision    recall  f1-score   support

       happy       0.80      0.72      0.76      2297
       angry       0.65      0.65      0.65      2190
         sad       0.80      0.88      0.84      2210

    accuracy                           0.75      6697
   macro avg       0.75      0.75      0.75      6697
weighted avg       0.75      0.75      0.75      6697


Processing with raw extractor...
Processing 3813 files for emotions: ['angry' 'happy' 'sad']


...


Accuracy for synthetic - Emotion: 0.6614

Classification Report for synthetic - Emotion:
              precision    recall  f1-score   support

       happy       0.67      0.69      0.68      2271
       angry       0.55      0.51      0.53      2233
         sad       0.76      0.78      0.77      2283

    accuracy                           0.66      6787
   macro avg       0.66      0.66      0.66      6787
weighted avg       0.66      0.66      0.66      6787


Processing with privacy extractor...
Processing 3813 files for emotions: ['angry' 'happy' 'sad']


100%|██████████| 3813/3813 [02:52<00:00, 22.15it/s]



Accuracy for privacy - Emotion: 0.5829

Classification Report for privacy - Emotion:
              precision    recall  f1-score   support

       happy       0.64      0.54      0.58      2272
       angry       0.53      0.40      0.46      2272
         sad       0.58      0.82      0.68      2226

    accuracy                           0.58      6770
   macro avg       0.58      0.58      0.57      6770
weighted avg       0.58      0.58      0.57      6770


Processing with samosa extractor...
Processing 3813 files for emotions: ['angry' 'happy' 'sad']


100%|██████████| 3813/3813 [03:09<00:00, 20.15it/s]



Accuracy for samosa - Emotion: 0.6016

Classification Report for samosa - Emotion:
              precision    recall  f1-score   support

       happy       0.62      0.59      0.61      2271
       angry       0.51      0.42      0.46      2233
         sad       0.65      0.79      0.71      2283

    accuracy                           0.60      6787
   macro avg       0.59      0.60      0.59      6787
weighted avg       0.59      0.60      0.59      6787


=== ACCURACY SUMMARY ===

Kirigami:
  Emotion: 64.63%

Coughsense:
  Emotion: 74.85%

Raw:
  Emotion: 68.94%

Synthetic:
  Emotion: 66.14%

Privacy:
  Emotion: 58.29%

Samosa:
  Emotion: 60.16%
